In [11]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [12]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables
using Random

In [13]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
all_scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    all_scenarios[i] = df_helper
end

In [14]:
W = 250
hours = 24
Random.seed!(123)
selected_scenarios = rand(1:n_scenarios, W)

scenarios = Dict()
counter = 1
for i in selected_scenarios
    scenarios[counter] = all_scenarios[i]
    counter += 1
end

In [15]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W .*
(scenarios[i][hour,"price"] * p_DA[hour]
+ scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"])
+ 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])


# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

Optimal solution found
Objective value: 151705.65829751547


In [16]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))

Objective value: 151705.65829751547
p_DA: [200.0, 0.0, 200.0, 200.0, 0.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 0.0, 200.0, 0.0, 0.0, 0.0, 200.0]
delta: [-104.95483872580644 95.08322575806451 -97.75548388709676 -96.12451624193548 100.0898386935484 -99.76580651612905 -101.42157262096774 101.97451611290322 106.93258062903226 -88.71129038709678 108.81903216129032 104.60354829032256 -99.03161296774192 -102.03758075806452 -111.48951616129033 -121.32193553225807 -136.69016135483872 -155.3314516935484 32.1540160483871 -170.18558072580646 31.063806354838707 27.22990317741936 23.543806322580643 -175.49351619354837; -150.38096774193548 46.19774191935483 -154.88370969354838 -154.00967741935483 43.07419354838709 -160.02112904838708 -159.08693551612905 40.538709661290326 39.468709629032254 -163.50580646774193 29.257467564516134 21.153532177419358 -177.85303237096775 -179.62161306451614 -180.2993226935484 -180.89283879032257 -177.21779033870968 -167.79122

In [17]:
objective_value(model)

151705.65829751547

In [18]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])

# make a table with the results
for hour in 1:hours
    push!(result_df, [hour, value(p_DA[hour])])
end

profit_df = DataFrame(scenario = Int[], profit = Float64[])

for i in 1:W
    profit = 0
    for hour in 1:hours
        profit += (scenarios[i][hour,"price"] * value(p_DA[hour])
        + scenarios[i][hour,"price"] * value(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"])))
    end
    push!(profit_df, [i, profit])
end

In [19]:
pretty_table(result_df, backend = Val(:latex))

\begin{tabular}{rr}
  \hline
  \textbf{hour} & \textbf{p\_DA} \\
  \texttt{Int64} & \texttt{Float64} \\\hline
  1 & 200.0 \\
  2 & 0.0 \\
  3 & 200.0 \\
  4 & 200.0 \\
  5 & 0.0 \\
  6 & 200.0 \\
  7 & 200.0 \\
  8 & 0.0 \\
  9 & 0.0 \\
  10 & 200.0 \\
  11 & 0.0 \\
  12 & 0.0 \\
  13 & 200.0 \\
  14 & 200.0 \\
  15 & 200.0 \\
  16 & 200.0 \\
  17 & 200.0 \\
  18 & 200.0 \\
  19 & 0.0 \\
  20 & 200.0 \\
  21 & 0.0 \\
  22 & 0.0 \\
  23 & 0.0 \\
  24 & 200.0 \\\hline
\end{tabular}


In [20]:
CSV.write("1_1_results/profits_1_1.csv", profit_df)
CSV.write("1_1_results/market_results_1_1.csv", result_df)

"1_1_results/market_results_1_1.csv"